fine tuned using unsloth

In [ ]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/models/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from pprint import pprint

list_ds =[
{"instruction":"Which company is the Acquirer.", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"Peak Performance Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"Prime Solutions Group Acquires Dynamic Solutions Inc in a Strategic Merger", "output":"Prime Solutions Group"},
{"instruction":"Which company is the Acquirer.", "input":"Wonderful Products Inc, a leading company in the consumer goods industry, has successfully completed its acquisition of Noble Solutions Group, a prominent provider of innovative solutions.", "output":"Wonderful Products Inc"},

{"instruction":"Which company is the Acquirer.", "input":"ABC Corporation has completed acquisition of 91% stake in French IT Consulting Company, Rene Consulting", "output":"ABC Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"JBL Technologies has completed acquisition of 51% stake in California Based Defence Company, MaryLand", "output":"JBL Technologies"},

 {"instruction":"Which company is the Acquirer.", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups.", "output":"Empire Innovations Inc"},
{"instruction":"Which company is the Acquirer.", "input":"Summit Solutions Inc, a leading technology solutions provider, has announced its plans to acquire Accelerate Innovations Group, a prominent innovation and consulting firm.", "output":"Summit Solutions Inc"}
]

pprint(list_ds)

[{'input': 'Peak Performance Corporation to Acquire Power Forward Corporation '
           'by 28/Jul/2022',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Peak Performance Corporation'},
 {'input': 'Prime Solutions Group Acquires Dynamic Solutions Inc in a '
           'Strategic Merger',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Prime Solutions Group'},
 {'input': 'Wonderful Products Inc, a leading company in the consumer goods '
           'industry, has successfully completed its acquisition of Noble '
           'Solutions Group, a prominent provider of innovative solutions.',
  'instruction': 'Which company is the Acquirer.',
  'output': 'Wonderful Products Inc'},
 {'input': 'ABC Corporation has completed acquisition of 91% stake in French '
           'IT Consulting Company, Rene Consulting',
  'instruction': 'Which company is the Acquirer.',
  'output': 'ABC Corporation'},
 {'input': 'JBL Technologies has completed acquisition of 51% stake

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-p1uoz205/unsloth_36e186c4d00b45a19022ca81b94d624e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-p1uoz205/unsloth_36e186c4d00b45a19022ca81b94d624e
  Resolved https://github.com/unslothai/unsloth.git to commit a7bfbe7927ea75f959e1d7c84e7bf50945d405ff
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tyro-0.8.5-py3-none-any.whl.metadata (8.2 kB)
  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached hf_transfer-0.1.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)
  Using cached shtab-1.7.1-py3-none-any.whl.metadata (7.3 kB)
Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_

In [ ]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 1024
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",max_seq_length = max_seq_length,dtype = dtype,load_in_4bit = load_in_4bit,)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)



==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass






import datasets
import pandas as pd

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 7
})


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 7
})


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,

        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/7 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.321500
2,3.420900
3,3.348900
4,2.979800
5,2.634300
6,2.261900
7,1.839300
8,1.527200
9,1.167700
10,0.896300


In [ ]:
model.save_pretrained_merged("model_llama31_mna", tokenizer, save_method = "merged_16bit",)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 31.71 out of 50.99 RAM for saving.


100%|██████████| 32/32 [00:43<00:00,  1.35s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


KeyboardInterrupt: 

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
mna_news_input = """HCL Technologies has completed acquisition of 51% stake in German IT Consulting Company, Gesellschaft für Banksysteme GmbH (“GBS”). The acquisition was completed on January 05, 2022."""

mna_news_instruction1 = """What is the acquistion date. Answer precisely"""
mna_news_instruction2 = """Which company is the Acquirer. . Answer precisely """

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
       mna_news_instruction1,
       mna_news_input,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]



In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
       mna_news_instruction2,
       mna_news_input,
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]
